In [ ]:
from sentence_transformers import SentenceTransformer, util

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-distilbert-cos-v1')

In [11]:
print('banana')

banana


In [ ]:
query = "I just discovered the course. Can I still join it?"
docs = [""]

#Encode query and documents
query_emb = model.encode(query)
v = query_emb
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
#print(scores)

print ('Q1- What''s the first value of the resulting vector?')
print(query_emb[0])


In [ ]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [ ]:
#filter
fitered_ml_docs = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':        
        fitered_ml_docs.append(doc)



In [ ]:
#get embeddings_fitered_ml_docs
embeddings_fitered_ml_docs = []
for doc in fitered_ml_docs:
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings_fitered_ml_docs.append(model.encode(qa_text))

In [ ]:
#convert with numpy
import numpy as np
x=[]
x = np.array(embeddings_fitered_ml_docs)

print ('Q2- What''s the shape of X? (X.shape)')
print(x.shape)

In [ ]:
scores = x.dot(v)
max = np.max(scores)
print ('Q3 - What''s the highest score in the results?')
print(max)


In [ ]:
# np_emb_filtered_ml_docs = np.array(embeddings_fitered_ml_docs)
# banana= VectorSearchEngine(fitered_ml_docs, np_emb_filtered_ml_docs)
# maca = banana.search(query_emb, num_results=5)
# maca

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
gt_question_embeddings = []


In [ ]:
#crate an embeddings array with all the questions from gt
for doc in tqdm(ground_truth):
    q = doc['question']
    q_v = model.encode(q)
    doc['vquestion']=q_v
    gt_question_embeddings.append(q_v)

In [ ]:
import numpy as np
#convert to nparray (needed for the search)
np_queston_emb=[]
np_queston_emb = np.array(gt_question_embeddings)

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


#pass the original documents and the emb_questions nparray 
search_engine = VectorSearchEngine(ground_truth, np_queston_emb)

In [ ]:

def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relevance_total)


def evaluateVSE(ground_truth, se):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results =  se(q['vquestion'], 5)
        relevance = [d['document'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total)
    }

hitrate = evaluateVSE(ground_truth, search_engine.search)
print ('Q4 - Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5./nWhat did you get?')
print(hitrate['hit_rate'])


In [2]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [5]:
#es_client = Elasticsearch('http://localhost:9200')
es_client = Elasticsearch(hosts=['http://172.18.0.2:9200'], 
                          max_retries=10, 
                          retry_on_timeout=True,
                          basic_auth=('elastic', 'bbvapMWzvbcykMLn1GkR')) 


In [7]:
#es_client = Elasticsearch('http://localhost:9200')
es_client = Elasticsearch(hosts=['http://172.18.0.2:9200'], 
                          max_retries=10, 
                          retry_on_timeout=True) 

In [ ]:
response = es_client.info()
print(response)

In [ ]:
##JSON
# base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
# relative_url = '03-vector-search/eval/documents-with-ids.json'
# docs_url = f'{base_url}/{relative_url}?raw=1'
# docs_response = requests.get(docs_url)
# documents = docs_response.json()

In [ ]:
#CSV
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
for doc in tqdm(ground_truth):
    question = doc['question']
    # text = doc['text']
    # q_a = question + ' ' + text

    doc['vquestion'] = model.encode(question)
    # doc['vtext'] = model.encode(text)
    # doc['vqa'] = model.encode(q_a)



In [9]:
import os
es_host = os.environ.get('ES_HOST', '172.18.0.2')  # Replace with the IP address of your Elasticsearch container
es_port = 9200

es_client = Elasticsearch(
    hosts=[f'http://{es_host}:{es_port}'],
    max_retries=10,
    retry_on_timeout=True
)

response = es_client.info()
print(response)


KeyboardInterrupt: 

In [ ]:
#es_client = Elasticsearch('http://localhost:9200')
es_client = Elasticsearch(hosts=['http://elastic:9200'], 
                          max_retries=10, 
                          retry_on_timeout=True,
                          basic_auth=('elastic', 'bbvapMWzvbcykMLn1GkR')) 

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "vquestion": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "vtext": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "vqa": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-question"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(ground_truth):
    es_client.index(index=index_name,document= doc)

In [ ]:

# #simple ES query POC
# q = ''
# v_q = query_emb

# queryParams = {
#      "field": "vquestion",
#      "query_vector": v_q,
#      "k": 5,
#      "num_candidates": 1000
# }


# es_result = es_client.search(
#     index=index_name, 
#     knn=queryParams,
#     source=["question", "course", "document"]
# )

# print('Total:', len(es_result['hits']['hits']))
# es_result['hits']['hits']

In [ ]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'vquestion') + 1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["document", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["question", "course", "document", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs



In [ ]:
results = elastic_search_knn('vquestion',query_emb, 'machine-learning-zoomcamp')
print ('Q5 - What''s the ID of the document with the highest score?')
print(results[0]['document'])


In [ ]:
def es_search(q):
    #question=q['question']
    course = q['course']
    #v_q = model.encode(question)
    return elastic_search_knn('vquestion', q['vquestion'], course)


def evaluate2(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results =  search_function(q)
        relevance = [d['document'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total)
    }

evaluate2(ground_truth, es_search)

In [ ]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']

    doc['vquestion'] = model.encode(question)
    doc['vtext'] = model.encode(text)

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document= doc)

In [ ]:
def elastic_search_knn2(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["question", "course", "id", "section"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs



results = elastic_search_knn2('vquestion',query_emb, 'data-engineering-zoomcamp')
results

In [ ]:
def evaluate3(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results =  search_function(ground_truth[0])
        relevance = [d['document'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total)
    }

#evaluate3(ground_truth, es_search)
es_search(ground_truth[0])